In [86]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix

warnings.filterwarnings("ignore")

train = pd.read_csv("train.csv", index_col='id')
train.head()


,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,,
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [87]:
train.columns = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Whole_weight_1', 'Whole_weight_2', 'Shell_weight', 'Rings']



sex = LabelEncoder().fit_transform(train.Sex)
train = train.drop('Sex', axis=1)
train['Sex'] = sex
train

,Length,Diameter,Height,Whole_weight,Whole_weight_1,Whole_weight_2,Shell_weight,Rings,Sex
id,,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,1
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,2
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,1
...,...,...,...,...,...,...,...,...,...
90610,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,6,2
90611,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,9,2
90612,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,6,1


In [88]:
# def log_transformation(data, columns):
#     for column in columns:
#         positive_values = data[column] - data[column].min() + 1
#         data[f'{column}_log'] = np.log(positive_values)
#     return data

# train = log_transformation(train, ['Length', 'Diameter', 'Height', 'Whole_weight', 'Whole_weight_1', 'Whole_weight_2', 'Shell_weight'])

In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Convert Rings column to categorical
train['Rings'] = train['Rings'].astype('category')

# Split the data into features and target variable
X = train.drop(columns=['Rings'])
y = train['Rings']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the target variable
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize the XGBoost classifier
model = XGBClassifier()

# Define hyperparameters for tuning
param_grid = {
    'max_depth': [3, 5 ],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 300, 500]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Initialize the XGBoost classifier with the best parameters
best_model = XGBClassifier(**best_params)

# Fit the best model on the training data
best_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


KeyboardInterrupt: 